# ETL and ingestion of data

In [ ]:
import os
import re
import json


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

COLOR = 'white'
plt.rcParams['text.color'       ] = COLOR
plt.rcParams['text.color'       ] = COLOR
plt.rcParams['axes.labelcolor'  ] = COLOR
plt.rcParams['xtick.color'      ] = COLOR
plt.rcParams['ytick.color'      ] = COLOR

# IPM data - December 2021 Scrape

In [ ]:
_PATH = Path('../data/uc-ipm/scrape_cleaned_Dec2021')
DATA_FILE_NAMES = sorted(_PATH.iterdir())
[data_file.name for data_file in DATA_FILE_NAMES]

The list of files should be as following:
```python
['exoticPests.json',
 'fruitItems_new.json',
 'fruitVeggieEnvironItems_new.json',
 'pestDiseaseItems_new.json',
 'plantFlowerItems.json',
 'turfPests.json',
 'veggieItems_new.json',
 'weedItems.json']
```

The corresponding ETL for these sources (links):
* [`exoticPests.json`](#exoticpestsjson)
* [`fruitItems_new.json`](#fruititems_newjson)
* [`fruitVeggieEnvironItems_new.json`](#fruitveggieenvironitems_newjson)
* [`pestDiseaseItems_new.json`](#pestdiseaseitems_newjson)
* [`plantFlowerItems.json`](#plantfloweritemsjson)
* [`turfPests.json`](#turfpestsjson)
* [`veggieItems_new.json`](#veggieitems_newjson)
* [`weedItems.json`](#weeditemsjson)

## ETL of data

In [ ]:
finalDf = pd.DataFrame()
cols = ['source', 'url', 'title', 'description', 'identification', 'development', 'damage', 'management', 'links']

### `exoticPests.json`
<a id='exoticpestsjson'></a>

In [ ]:
def pestsExotic():
    # -------------------------------------------- Exotic pests
    print(f'Merging exotic pests...')
    FILE_NAME = 'exoticPests.json'
    df = pd.read_json(Path.joinpath(_PATH, FILE_NAME))
    '''
    columns in source:
    name url description damage identification life_cycle monitoring management related_links images
    final schema:
    source url title description identification development damage management links
    '''

    df['source'] = 'ucipm'

    df.rename(columns = {
        'name'      : 'title'       ,
        'life_cycle': 'development' ,
        'images'    : 'links'       ,
    }, inplace = True)

    df['links'] = df['links'].apply(lambda d: d if isinstance(d, list) else [])

    df['links'] = df.apply(lambda r: [
        {
            'type'  : 'image'   , 
            'src'   : i['src']  , 
            'link'  : i['link'] ,
            'title' : r['title'] + ' - ' + i['caption']
        } for i in r['links'] if len(i['caption']) > 0], axis = 1)
    
    df['related_links'] = df.apply(lambda r: [
        {
            'type'  : 'page'   , 
            'src'   : i['link'] , 
            'link'  : ''        ,
            'title' : r['title'] + ' - ' + i['text']
        } for i in r['related_links'] if len(i['text']) > 0], axis = 1)

    df.apply(lambda x: x['links'].extend(x['related_links' ]), axis = 1)

    df = df[cols]

    return df

### `fruitItems_new.json`
<a id='fruititems_newjson'></a>

In [ ]:
def infoFruits():
    # -------------------------------------------- Fruits information
    print(f'Merging fruits information...')
    FILE_NAME = 'fruitItems_new.json'
    df = pd.read_json(Path.joinpath(_PATH, FILE_NAME))
    '''
    columns in source:
    name url cultural_tips pests_and_disorders
    final schema:
    source url title description identification development damage management links
    '''

    df['source'         ] = 'ucipm'
    df['description'    ] = ''
    df['identification' ] = ''
    df['development'    ] = ''
    df['damage'         ] = ''
    df['management'     ] = ''
    
    df.rename(columns = {
        'name'          : 'title',
        'cultural_tips' : 'links'
    }, inplace = True)

    df['links'] = df['links'].apply(lambda d: d if isinstance(d, list) else [])

    df['links'] = df.apply(lambda r: [
        {
            'type'  : 'tip'     , 
            'src'   : i['link'] , 
            'link'  : ''        ,
            'title' : r['title'] + ' - ' + i['tip']
        } for i in r['links'] if len(i['tip']) > 0], axis = 1)

    df['pests_and_disorders'] = df.apply(lambda r: [
        {
            'type'  : 'problem' , 
            'src'   : i['link'] , 
            'link'  : ''        ,
            'title' : r['title'] + ' - ' + i['problem']
        } for i in r['pests_and_disorders'] if len(i['problem']) > 0], axis = 1)
    
    df.apply(lambda x: x['links'].extend(x['pests_and_disorders' ]), axis = 1)

    df = df[cols]
    return df

### `fruitVeggieEnvironItems_new.json`
<a id='fruitveggieenvironitems_newjson'></a>

In [ ]:
def damagesEnvironment():
    # -------------------------------------------- Fruit and veggie damages
    print(f'Merging fruit and veggie damages...')
    FILE_NAME = 'fruitVeggieEnvironItems_new.json'
    df = pd.read_json(Path.joinpath(_PATH, FILE_NAME))
    '''
    columns in source:
    name url description identification damage disorder_development solutions images
    final schema:
    source url title description identification development damage management links
    '''

    df['source'] = 'ucipm'

    df.rename(columns = {
        'name'                  : 'title'       ,
        'disorder_development'  : 'development' ,
        'solutions'             : 'management' ,
        'images'                : 'links'      ,
    }, inplace = True)

    df['links'] = df.apply(lambda r: [
        {
            'type'  : 'image'   , 
            'src'   : i['src']  , 
            'link'  : i['link'] ,
            'title' : r['title'] + ' - ' + i['caption']
        } for i in r['links'] if len(i['caption']) > 0], axis = 1)

    df = df[cols]

    return df

### `pestDiseaseItems_new.json`
<a id='pestdiseaseitems_newjson'></a>

In [ ]:
def pestsDiseases():
    # -------------------------------------------- Pests diseases
    print(f'Merging pests diseases...')
    FILE_NAME = 'pestDiseaseItems_new.json'
    df = pd.read_json(Path.joinpath(_PATH, FILE_NAME))
    '''
    columns in source:
    name url description identification life_cycle damage solutions images
    final schema:
    source url title description identification development damage management links
    '''

    df['source'] = 'ucipm'

    df.rename(columns = {
        'name'          : 'title'       ,
        'life_cycle'    : 'development' ,
        'damagePestNote': 'damage'      ,
        'solutions'     : 'management'  ,
        'images'        : 'links'       ,
    }, inplace = True)


    df['links'] = df.apply(lambda r: [
        {
            'type'  : 'image'   , 
            'src'   : i['src']  , 
            'link'  : i['link'] ,
            'title' : r['title'] + ' - ' + i['caption']
        } for i in r['links'] if len(i['caption']) > 0], axis = 1)

    df = df[cols]

    return df


### `plantFlowerItems.json`
<a id='plantfloweritemsjson'></a>

In [ ]:
def infoFlowers():
    # -------------------------------------------- Flowers information
    print(f'Merging flowers information...')
    FILE_NAME = 'plantFlowerItems.json'
    df = pd.read_json(Path.joinpath(_PATH, FILE_NAME))
    '''
    columns in source:
    name url identification optimum_conditions pests_and_disorders images
    final schema:
    source url title description identification development damage management links
    '''

    df['source'         ] = 'ucipm'
    df['description'    ] = ''
    df['development'    ] = ''
    df['damage'         ] = ''

    df.rename(columns = {
        'name'              : 'title'       ,
        'optimum_conditions': 'management'  ,
        'images'            : 'links'       ,
    }, inplace = True)

    df['links'] = df.apply(lambda r: [
        {
            'type'  : 'image'   , 
            'src'   : i['src']  , 
            'link'  : i['link'] ,
            'title' : r['title'] + ' - ' + i['caption']
        } for i in r['links'] if len(i['caption']) > 0], axis = 1)

    df['pests_and_disorders'] = df.apply(lambda r: [
        {
            'type'  : 'problem' , 
            'src'   : i['link'] , 
            'link'  : ''        ,
            'title' : r['title'] + ' - ' + i['problem']
        } for i in r['pests_and_disorders'] if len(i['problem']) > 0], axis = 1)

    df.apply(lambda x: x['links'].extend(x['pests_and_disorders' ]), axis = 1)

    df = df[cols]

    return df

### `turfPests.json`
<a id='plantfloweritemsjson'></a>

In [ ]:
def pestsTurf():
    # -------------------------------------------- Turf pests
    print(f'Merging turf pests...')
    FILE_NAME = 'turfPests.json'
    df = pd.read_json(Path.joinpath(_PATH, FILE_NAME))
    '''
    columns in source:
    name url text images
    final schema:
    source url title description identification development damage management links
    '''

    df['source'         ] = 'ucipm'
    df['identification' ] = ''
    df['development'    ] = ''
    df['damage'         ] = ''
    df['management'     ] = ''

    df.rename(columns = {
        'name'  : 'title'       ,
        'text'  : 'description' ,
        'images': 'links'       ,
    }, inplace = True)


    df['links'] = df.apply(lambda r: [
        {
            'type'  : 'image'   , 
            'src'   : i['src']  , 
            'link'  : i['link'] ,
            'title' : r['title'] + ' - ' + i['caption']
        } for i in r['links'] if len(i['caption']) > 0], axis = 1)

    df = df[cols]

    return df

### `veggieItems_new.json`
<a id='veggieitems_newjson'></a>

In [ ]:
def infoVeggies():
    # -------------------------------------------- Veggies information
    print(f'Merging veggies information...')
    FILE_NAME = 'veggieItems_new.json'
    df = pd.read_json(Path.joinpath(_PATH, FILE_NAME))
    '''
    columns in source:
    name url description tips images pests_and_disorders
    final schema:
    source url title description identification development damage management links
    '''

    df['source'         ] = 'ucipm'
    df['identification' ] = ''
    df['development'    ] = ''
    df['damage'         ] = ''
    
    df.rename(columns = {
        'name'  : 'title'       ,
        'tips'  : 'management'  ,
        'images': 'links'       ,
    }, inplace = True)

    df['links'] = df.apply(lambda r: [
        {
            'type'  : 'image'   , 
            'src'   : i['src']  , 
            'link'  : i['link'] ,
            'title' : r['title'] + ' - ' + i['caption']
        } for i in r['links'] if len(i['caption']) > 0], axis = 1)

    df['pests_and_disorders'] = df.apply(lambda r: [
        {
            'type'  : 'problem' , 
            'src'   : i['link'] , 
            'link'  : ''        ,
            'title' : r['title'] + ' - ' + i['problem']
        } for i in r['pests_and_disorders'] if len(i['problem']) > 0], axis = 1)

    df.apply(lambda x: x['links'].extend(x['pests_and_disorders' ]), axis = 1)

    df = df[cols]

    return df

### `weedItems.json`
<a id='weeditemsjson'></a>

In [ ]:
def damagesWeed():
    # -------------------------------------------- Weed damages
    print(f'Merging weed damages...')
    FILE_NAME = 'weedItems.json'
    df = pd.read_json(Path.joinpath(_PATH, FILE_NAME))
    '''
    columns in source:
    name url description images
    final schema:
    source url title description identification development damage management links
    '''

    df['source'         ] = 'ucipm'
    df['identification' ] = ''
    df['development'    ] = ''
    df['damage'         ] = ''
    df['management'     ] = ''

    df.rename(columns = {
        'name'  : 'title',
        'images': 'links'
    }, inplace = True)

    df['links'] = df.apply(lambda r: [
        {
            'type'  : 'image'   , 
            'src'   : i['link'] , 
            'link'  : ''        ,
            'title' : r['title'] + ' - ' + i['caption']
        } for i in r['links'] if len(i['caption']) > 0], axis = 1)

    df = df[cols]

    return df

### Merging and transforming

In [ ]:
df      = pestsExotic()
finalDf = pd.concat([finalDf, df], ignore_index = True, axis = 0)
print(f'Final dataframe shape: {finalDf.shape}')
print(f'------------------------------------------------')

df      = infoFruits()
finalDf = pd.concat([finalDf, df], ignore_index = True, axis = 0)
print(f'Final dataframe shape: {finalDf.shape}')
print(f'------------------------------------------------')

df      = damagesEnvironment()
finalDf = pd.concat([finalDf, df], ignore_index = True, axis = 0)
print(f'Final dataframe shape: {finalDf.shape}')
print(f'------------------------------------------------')

df      = pestsDiseases()
finalDf = pd.concat([finalDf, df], ignore_index = True, axis = 0)
print(f'Final dataframe shape: {finalDf.shape}')
print(f'------------------------------------------------')


df      = infoFlowers()
finalDf = pd.concat([finalDf, df], ignore_index = True, axis = 0)
print(f'Final dataframe shape: {finalDf.shape}')
print(f'------------------------------------------------')

df      = pestsTurf()
finalDf = pd.concat([finalDf, df], ignore_index = True, axis = 0)
print(f'Final dataframe shape: {finalDf.shape}')
print(f'------------------------------------------------')

df      = infoVeggies()
finalDf = pd.concat([finalDf, df], ignore_index = True, axis = 0)
print(f'Final dataframe shape: {finalDf.shape}')
print(f'------------------------------------------------')

df      = damagesWeed()
finalDf = pd.concat([finalDf, df], ignore_index = True, axis = 0)
print(f'Final dataframe shape: {finalDf.shape}')
print(f'------------------------------------------------')

def _clean(text):
    '''
    Fix encodings and remove escape and redundant whitespace characters from text.
    '''
    text = text.encode('ascii', 'ignore').decode()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

colsVector = ['title', 'description', 'identification', 'development', 'damage', 'management']
for c in colsVector:
    finalDf[c] = finalDf[c].apply(_clean)

print(f'Fix encodings and remove escape and redundant whitespace characters from text.')
print(f'------------------------------------------------')

print(f'Final dataframe shape: {finalDf.shape    }')
print(f'FINISHED')

finalDf.sample(5)

# IPM data - April 2022 Scrape

In [ ]:
_PATH = Path('../data/uc-ipm/scrape_cleaned_Apr2022/')
DATA_FILE_NAMES = sorted(_PATH.iterdir())
[data_file.name for data_file in DATA_FILE_NAMES]

The list of files should be as following:
```python
['FruitVegCulturalItems.json',
 'GardenControlsPestItems.json',
 'GardenControlsPesticideItems.json',
 'PestNotes.json',
 'QuickTips.json',
 'Videos.json',
 'WeedIdItems.json']
```

The corresponding ETL for these sources (links):
* [`FruitVegCulturalItems.json`](#fruitvegculturalitemsjson)
* [`GardenControlsPestItems.json`](#gardercontolspestitemsjson)
* [`GardenControlsPesticideItems.json`](#gardencontrolspesticideitemsjson)
* [`PestNotes.json`](#pestnotesjson)
* [`QuickTips.json`](#quicktipsjson)
* [`Videos.json`](#videosjson)
* [`WeedIdItems.json`](#weediditemsjson)

## ETL of data

In [ ]:
cols = ['source', 'url', 'title', 'description', 'identification', 'development', 'damage', 'management', 'links']

### `FruitVegCulturalItems.json`
<a id='fruitvegculturalitemsjson'></a>

In [ ]:
def infoFruitVegCultural():
    # -------------------------------------------- Fruit and veggie cultural tips
    print(f'Merging fruit and veggie cultural tips..')
    FILE_NAME = 'FruitVegCulturalItems.json'
    df = pd.read_json(Path.joinpath(_PATH, FILE_NAME))
    '''
    columns in source:
    name url description images tips_table
    final schema:
    source url title description identification development damage management links
    '''

    df['source'         ] = 'ucipm'        
    df['identification' ] = ''
    df['development'    ] = ''
    df['damage'         ] = ''
    df['management'     ] = ''

    df.rename(columns = {
        'name'  : 'title',
        'images': 'links'
    }, inplace = True)

    df['links'] = df.apply(lambda r: [
        {
            'type'  : 'image'   , 
            'src'   : i['src']  , 
            'link'  : ''        ,
            'title' : r['title'] + ' - ' + i['caption']
        } for i in r['links'] if len(i['caption']) > 0], axis = 1)

    df['tips_table'] = df.apply(lambda r: [
        {
            'type'  : 'problem' , 
            'src'   : r['url'] , 
            'link'  : ''        ,
            'title' : r['title'] + ' - ' + i['header']
        } for i in r['tips_table'] if 'header' in i and len(i['header']) > 0], axis = 1)

    df.apply(lambda x: x['links'].extend(x['tips_table' ]), axis = 1)

    df = df[cols]
    
    return df

### `GarderContolsPestItems.json`
<a id='gardercontolspestitemsjson'></a>

In [ ]:
def infoPestControl():
    # -------------------------------------------- Garden pest control
    print(f'Merging garden pest control information...')
    FILE_NAME = 'GardenControlsPestItems.json'
    df = pd.read_json(Path.joinpath(_PATH, FILE_NAME))
    '''
    columns in source:
    name url description images
    final schema:
    source url title description identification development damage management links
    '''

    df['source'         ] = 'ucipm'
    df['identification' ] = ''
    df['development'    ] = ''
    df['damage'         ] = ''
    df['management'     ] = ''

    df.rename(columns = {
        'name'  : 'title',
        'images': 'links'
    }, inplace = True)

    df['links'] = df['links'].apply(lambda d: d if isinstance(d, list) else [])

    df['links'] = df.apply(lambda r: [
        {
            'type'  : 'image'                , 
            'src'   : i.get('src'       , ''), 
            'link'  : i.get('link'      , ''),
            'title' : r['title'] + ' - ' + i['caption']
        } for i in r['links'] if len(i['caption']) > 0], axis = 1)

    df = df[cols]
    
    return df

### `GardenControlsPesticideItems.json`
<a id='gardencontrolspesticideitemsjson'></a>

In [ ]:
def infoPesticideControl():
    # -------------------------------------------- Garden pesticide control
    print(f'Merging garden pesticide control information...')
    FILE_NAME = 'GardenControlsPesticideItems.json'
    df = pd.read_json(Path.joinpath(_PATH, FILE_NAME))
    '''
    columns in source:
    active_ingredient url pesticide_type information
    final schema:
    source url title description identification development damage management links
    '''

    df['source'         ] = 'ucipm'
    df['identification' ] = ''
    df['development'    ] = ''
    df['damage'         ] = ''
    df['management'     ] = ''
    
    df['title'          ] = df[['active_ingredient', 'pesticide_type']].agg(' - '.join, axis=1)
    df['description'    ] = df['information'].str[0].apply(lambda x: x['associated_pests'])
    df['links'          ] = [[] for _ in range(len(df))]

    df = df[cols]

    return df

### `PestNotes.json`
<a id='pestnotesjson'></a>

In [ ]:
def pestsNotes():
    # -------------------------------------------- Pests IPM
    print(f'Merging pests notes...')
    FILE_NAME = 'PestNotes.json'
    df = pd.read_json(Path.joinpath(_PATH, FILE_NAME))
    '''
    columns in source:
    name urlPestNote descriptionPestNote lifecyclePestNote damagePestNote managementPestNote imagePestNote tablePestNote
    final schema:
    source url title description identification development damage management links
    '''

    df['source'         ] = 'ucipm'
    df['identification' ] = ''

    df.rename(columns = {
        'urlPestNote'           : 'url'         ,
        'name'                  : 'title'       ,
        'descriptionPestNote'   : 'description' ,
        'lifecyclePestNote'     : 'development' ,
        'damagePestNote'        : 'damage'      ,
        'managementPestNote'    : 'management'  ,
        'imagePestNote'         : 'links'       ,
    }, inplace = True)

    df['links'] = df.apply(lambda r: [
        {
            'type'  : 'image'   , 
            'src'   : i['src']  , 
            'link'  : i['link'] ,
            'title' : r['title'] + ' - ' + i['caption']
        } for i in r['links'] if len(i['caption']) > 0], axis = 1)

    df = df[cols]
    
    return df

### `QuickTips.json`
<a id='quicktipsjson'></a>

In [ ]:
def pestsQuickTips():
    # -------------------------------------------- Quick tips on pests
    print(f'Merging pests quick notes...')
    FILE_NAME = 'QuickTips.json'
    df = pd.read_json(Path.joinpath(_PATH, FILE_NAME))
    '''
    columns in source:
    name urlQuickTip contentQuickTips imageQuickTips
    final schema:
    source url title description identification development damage management links
    '''

    df['source'        ] = 'ucipm'
    df['identification'] = ''
    df['development'   ] = ''
    df['damage'        ] = ''
    df['management'    ] = ''

    df.rename(columns = {
        'urlQuickTip'           : 'url'         ,
        'name'                  : 'title'       ,
        'contentQuickTips'      : 'description' ,
        'imageQuickTips'        : 'links'       ,
    }, inplace = True)

    df['links'] = df.apply(lambda r: [
        {
            'type'  : 'image'   , 
            'src'   : i['src']  , 
            'link'  : i['link'] ,
            'title' : r['title'] + ' - ' + i['caption']
        } for i in r['links'] if len(i['caption']) > 0], axis = 1)

    df = df[cols]
    
    return df

### `Videos.json`
<a id='videosjson'></a>

In [ ]:
def pestsVideos():
    # -------------------------------------------- Videos of UC IPM YouTube data
    print(f'Merging UC IPM YouTube data...')
    FILE_NAME = 'Videos.json'
    df = pd.read_json(Path.joinpath(_PATH, FILE_NAME))
    '''
    columns in source:
    title url description
    final schema:
    source url title description identification development damage management links
    '''

    df['source'         ] = 'ucipm'
    df['identification' ] = ''
    df['development'    ] = ''
    df['damage'         ] = ''
    df['management'     ] = ''

    df['links'          ] = [[] for _ in range(len(df))]

    df = df[cols]

    return df

### `WeedIdItems.json`
<a id='weediditemsjson'></a>

In [ ]:
def pestsWeed():
    # -------------------------------------------- Weed related pests
    print(f'Merging weed related pests...')
    FILE_NAME = 'WeedIdItems.json'
    df = pd.read_json(Path.joinpath(_PATH, FILE_NAME))
    '''
    columns in source:
    name url description images
    final schema:
    source url title description identification development damage management links
    '''

    df['source'         ] = 'ucipm'
    df['identification' ] = ''
    df['development'    ] = ''
    df['damage'         ] = ''
    df['management'     ] = ''

    df.rename(columns = {
        'name'  : 'title',
        'images': 'links'
    }, inplace = True)

    df['links'] = df.apply(lambda r: [
        {
            'type'  : 'image'   , 
            'src'   : i['src']  , 
            'link'  : ''        ,
            'title' : r['title'] + ' - ' + i['caption']
        } for i in r['links'] if len(i['caption']) > 0], axis = 1)

    df = df[cols]

    return df

### Merging and transforming

In [ ]:
df      = infoFruitVegCultural()
finalDf = pd.concat([finalDf, df], ignore_index = True, axis = 0)
print(f'Final dataframe shape: {finalDf.shape}')
print(f'------------------------------------------------')

df      = infoPestControl()
finalDf = pd.concat([finalDf, df], ignore_index = True, axis = 0)
print(f'Final dataframe shape: {finalDf.shape}')
print(f'------------------------------------------------')

df      = infoPesticideControl()
finalDf = pd.concat([finalDf, df], ignore_index = True, axis = 0)
print(f'Final dataframe shape: {finalDf.shape}')
print(f'------------------------------------------------')

df      = pestsNotes()
finalDf = pd.concat([finalDf, df], ignore_index = True, axis = 0)
print(f'Final dataframe shape: {finalDf.shape}')
print(f'------------------------------------------------')

df      = pestsQuickTips()
finalDf = pd.concat([finalDf, df], ignore_index = True, axis = 0)
print(f'Final dataframe shape: {finalDf.shape}')
print(f'------------------------------------------------')

df      = pestsVideos()
finalDf = pd.concat([finalDf, df], ignore_index = True, axis = 0)
print(f'Final dataframe shape: {finalDf.shape    }')
print(f'------------------------------------------------')

df      = pestsWeed()
finalDf = pd.concat([finalDf, df], ignore_index = True, axis = 0)
print(f'Final dataframe shape: {finalDf.shape}')
print(f'------------------------------------------------')

def _clean(text):
    '''
    Fix encodings and remove escape and redundant whitespace characters from text.
    '''
    text = text.encode('ascii', 'ignore').decode()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

colsVector = ['title', 'description', 'identification', 'development', 'damage', 'management']
for c in colsVector:
    finalDf[c] = finalDf[c].apply(_clean)

print(f'Fix encodings and remove escape and redundant whitespace characters from text.')
print(f'------------------------------------------------')

print(f'Final dataframe shape: {finalDf.shape    }')
print(f'FINISHED')

finalDf.sample(5)

# AskExtension Data

In [ ]:
_PATH = Path('../data/askextension_kb/')
DATA_FILE_NAMES = sorted(_PATH.iterdir())

print(f'List of files:\n{[data_file.name for data_file in DATA_FILE_NAMES]}')

with open(DATA_FILE_NAMES[0]) as f:
    f = json.load(f)
    print(json.dumps(f[0], indent = 2))

The list of files should be as following:
```python
['2012-2013.json', '2014-2015.json', '2016-2017.json', '2018-2019.json', '2020-1.json', '2020-2.json', '2021-1.json', '2021-2.json']
```

__NB__: We will only using tickets from California state.

## ETL

In [ ]:
import sys
import re

from string import punctuation as pn

# Modify STATE_FILTER and MIN_WORD_COUNT variables accordingly
STATE_FILTER    = ['California']
MIN_WORD_COUNT  = 3

ASKEXTENSION_QUESTION_URL = 'https://ask2.extension.org/kb/faq.php?id='

# Combines the data files into one and returns it.
df = pd.DataFrame()
for f in DATA_FILE_NAMES:
    df = pd.concat([df, pd.read_json(f)], ignore_index = True, axis = 0)

df['source'] = 'ae_kb'

# Convert 'faq-id' to str type
df['faq-id'] = df['faq-id'].astype(str)

# Leave tickets from California state
df = df[df['state'].isin(STATE_FILTER)]

# Add the URL and leave blank URL for questions with no ID
df['url'] = [
    f"{ASKEXTENSION_QUESTION_URL}{ticket_no}" if len(ticket_no) == 6 else ""
    for ticket_no in df['title'].str.split('#').str[-1]
]

# Add the ticket number from title and leave blank for questions without
df['ticket-no'] = [
    ticket_no if len(ticket_no) == 6 else ""
    for ticket_no in df['title'].str.split('#').str[-1]
]

df.rename(columns = {'faq-id': 'faq_id', 'ticket-no': 'ticket_no'}, inplace = True)

def _clean(text):
    '''
    Fix encodings and remove escape and redundant whitespace characters from text.

    Examples with non-ascii characters - 110358, 147160
    Examples with redundant whitespace - 117069, 127760

    See: https://stackoverflow.com/a/53821967/5480536
    '''
    text = text.encode('ascii', 'ignore').decode()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def _transform_answer(answer_dict):
    '''
    Convert answer field from a dictionary to a list.
    '''
    answers = [{}] * len(answer_dict)
    
    for k, v in answer_dict.items():
        # clean the response up
        v = {
            'type'  : 'answer'  ,
            'src'   : ''        , 
            'link'  : ''        ,
            'title' : _clean(v['response']),
        }
        answers[int(k) - 1] = v
    
    return answers

# Transform answer for consistency with IPM data
df['links'] = df['answer'].apply(_transform_answer)
df['links'] = df.apply(lambda r: [
        {
            'type'  : i['type'] , 
            'src'   : r['url']  , 
            'link'  : ''        ,
            'title' : i['title']
        } for i in r['links'] if len(i['title']) > 0], axis = 1)

# Strip all spaces and remove non-ascii characters from text fields
for column in ['state', 'title', 'question']:
    df[column] = df[column].apply(_clean)

def _transform_title(title):
    '''
    Remove question ID from title, and append '.' in the end
    if no punctuation was detected.

    Example with '#' - 437259
    Example with '...' - 437264
    '''
    title = ''.join(title.split('#')[:-1]).strip().strip('...')
    
    # add a '.' if it does not yet end with a punctuation
    title = title if (title and title[-1] in pn) else title + '.'
    
    return title

# Clean ID and '...' from title, and append punctuation if not present
df['title'] = df['title'].apply(_transform_title)

def _merge_title_question(df):
    '''
    Create new column from questions and title,
    but only if it is not already exactly in the question.
    '''
    titles      = df['title'    ].tolist()
    questions   = df['question' ].tolist()
    
    tqs = [
        question
        if (title and question.startswith(title[:-1]))
        else title + " " + question
        for (title, question) in zip(titles, questions)
    ]

    return tqs

# Create new column from `title` and `question`, or only question
# if title is exactly the question     
df['description'] = _merge_title_question(df)
    
# Remove questions with small number words in title-question
if MIN_WORD_COUNT:
    df = df[df['description'].str.split().str.len() > MIN_WORD_COUNT]

df = df.loc[:, ['source', 'url', 'title', 'description', 'links']]
df.sample(5)

cols = ['source', 'url', 'title', 'description', 'identification', 'development', 'damage', 'management', 'links']
'''
columns in source:
source url name description links
final schema:
source url title description identification development damage management links    
'''
df['identification' ] = ''
df['development'    ] = ''
df['damage'         ] = ''
df['management'     ] = ''
df = df[cols]

finalDf = pd.concat([finalDf, df], ignore_index = True, axis = 0)
print(f'Final dataframe shape: {finalDf.shape}')
print(f'------------------------------------------------')

finalDf.sample(5)

# Embedding text fields into vectors and stripping text fields for saving into ES

In [ ]:
import sys

sys.path.insert(1, os.path.realpath(os.path.pardir))

os.environ['STAGE'          ] = 'dev'
os.environ['ES_USERNAME'    ] = 'elastic'
os.environ['ES_PASSWORD'    ] = 'changeme'
os.environ['TF_CACHE_DIR'   ] = '/var/tmp/models'
## select the environment for ingestion
os.environ['ES_HOST'    ] = 'http://localhost:9200/'
# os.environ['ES_HOST'    ] = 'https://dev.es.chat.ask.eduworks.com/'
# os.environ['ES_HOST'    ] = 'https://qa.es.chat.ask.eduworks.com/'

import config

In [ ]:
import importlib
importlib.reload(config)

### Transforming textual data

In [ ]:
CHUNK_SIZE      = 1
ROLLING_SIZE    = 3

colsVector = ['title', 'description', 'identification', 'development', 'damage', 'management']
print(f'Final DF: Transforming columns - {colsVector} and links titles.')

from spacy.lang.en import English 

nlp = English()
nlp.add_pipe('sentencizer')
raw_text = finalDf.iloc[0]['description']

print(f'STARTING TRANSFORMING')
c_items = []
for i, r in finalDf.iterrows():
    r_texts = []
    for c in colsVector:
        t = r[c]
        
        doc = nlp(t)
        
        ts = [sent for sent in doc.sents]
        if len(ts) == 0:
            continue
        else:
            chunks, chunk_size, roll_size = len(ts), CHUNK_SIZE, ROLLING_SIZE
            ts = [ts[i1:i1+chunk_size+(roll_size - 1)] for i1 in range(0, chunks - (roll_size - 1), chunk_size)]
            ts = [{'text': ' '.join([l2.text for l2 in l1]), 'name': c + '_' + str(i1), 'start': l1[0].start_char, 'end': l1[-1].end_char} for i1, l1 in enumerate(ts)]
                
        r_texts.extend(ts)
    
    ts = [r['title'] + ' - ' + i1['title'] for i1 in r['links']]
    if len(ts) == 0:
        c_items.append(r_texts)
        if (i+1) % 500 == 0:
            print(f'Finished transforming of {i+1} rows of dataframe')
        continue

    for i1, v in enumerate(ts):
        r_texts.append({'text': v, 'name': 'links_' + str(i1), 'start': 0, 'end': -1})
    
    c_items.append(r_texts)

    if (i+1) % 500 == 0:
        print(f'Finished transforming of {i+1} rows of dataframe')

print(f'Finished transforming of {i+1} rows of dataframe')
print(f'FINISHED TRANSFORMING')

### Checking for invalid links

In [ ]:
count_ucipm = 0
count_askextension = 0
for i, r in finalDf.iterrows():
    url = r['url']
    if len(r['url']) < 10:
        print(f'Source with no main link at row {i} of data frame, main link - {url}.')
    links = r['links']
    no_link = False
    show_main_url = False
    for l in links:
        if len(l['src']) < 10:
            no_link = True
            if not show_main_url:
                show_main_url = True
                print(f'Links at {url}')
            print(l)
    if no_link:
        if r['source'] == 'ae_kb':
            count_askextension += 1
        else:
            count_ucipm += 1
        

print(f'Number of sources from AskExtension with no link urls - {count_askextension}')
print(f'Number of sources from UC IPM with no link urls - {count_ucipm}')

### Embedding textual data

In [ ]:
BATCH_SIZE = 64

print(f'STARTING EMBEDDING')

finalDf['vectors'] = np.empty((len(finalDf), 0)).tolist()

# TF HUB model
# vectors   = config.embed(texts_modified).numpy().tolist()
    
# Sentence Encoder model        
vectors = config.embed.encode(
    sentences           = texts_modified,
    batch_size          = BATCH_SIZE    ,
    show_progress_bar   = True
).tolist()

index = 0
for i, r in enumerate(c_items):
    for i1, r1 in enumerate(r):
        r1['vector'] = vectors[index]
        r1.pop('text')
        index += 1

print(f'FINISHED EMBEDDING')

finalDf['vectors'] = c_items
print(f'The number of vectors to be ingested: {len([r1["vector"] for r in finalDf["vectors"] for r1 in r])}')        
finalDf.sample(5)

## Ingesting data into ES

In [ ]:
# Different embedding sizes depending on the models
# VECTOR_SIZE = 384
# VECTOR_SIZE = 512
VECTOR_SIZE = 768

mapping  = {
    "settings": {"number_of_shards": 2, "number_of_replicas": 1},
    "mappings": {
        "dynamic"   : "false",
        "_source"   : {"enabled": "true"},
        "properties": {
            "source"        : {"type": "keyword", "index": "true" , "ignore_above": 32766},
            "url"           : {"type": "keyword", "index": "false", "ignore_above": 32766},

            "title"         : {"type": "keyword", "index": "false", "ignore_above": 32766},
            "description"   : {"type": "keyword", "index": "false", "ignore_above": 32766},
            "identification": {"type": "keyword", "index": "false", "ignore_above": 32766},
            "development"   : {"type": "keyword", "index": "false", "ignore_above": 32766},
            "damage"        : {"type": "keyword", "index": "false", "ignore_above": 32766},
            "management"    : {"type": "keyword", "index": "false", "ignore_above": 32766},
            "vectors"       : {
                "type"      : "nested",
                "properties": {
                    "vector": {
                        "type": "dense_vector", 
                        "dims": VECTOR_SIZE
                    },
                    "name"  : {"type": "keyword", "index": "false", "ignore_above": 32766},
                    "start" : {"type": "integer"                                         },
                    "end"   : {"type": "integer"                                         },
                }
            },
            
            "links"         : {
                "type"      : "nested",
                "properties": {
                    "type"  : {"type": "keyword", "index": "false", "ignore_above": 32766},
                    "src"   : {"type": "keyword", "index": "false", "ignore_above": 32766},
                    "link"  : {"type": "keyword", "index": "false", "ignore_above": 32766},
                    "title" : {"type": "keyword", "index": "false", "ignore_above": 32766}
                }
            }
        }
    }
}

final_json = finalDf.to_dict('records')

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk

from collections import deque

# increase the timeout if necessary
es_client = Elasticsearch([config.es_host], http_auth=(config.es_username, config.es_password), timeout = 20)

es_client.indices.delete(
    index   = config.es_combined_index, 
    ignore  = 404)
es_client.indices.create(
    index       = config.es_combined_index  , 
    settings    = mapping['settings']       , 
    mappings    = mapping['mappings']       )
# play with chunk size parameter for timed out problem
deque(parallel_bulk(es_client, actions = final_json, index = config.es_combined_index, max_chunk_bytes = 5 * 1024 * 1024), maxlen = 0)

es_client.indices.refresh()